Scrape the Python ITJobsWatch page. Showcase:
1. Data Ingestion
2. Data Wrangling
3. Data Analysis
4. Data Visualisation

In [104]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

x = requests.get('https://www.itjobswatch.co.uk/jobs/uk/python.do')
soup = BeautifulSoup(x.text)

Parse a single table into pandas

In [105]:
def get_info_from_row(row):
    return [tag.get_text() for tag in row.find_all('td') if len(tag.get_text()) < 50]

In [106]:
def get_info_from_section(section):
    return [get_info_from_row(row) for row in section.find_all('tr') if len(get_info_from_row(row)) != 0]

In [107]:
def get_job_stats_df(soup):
    info = get_info_from_section(soup.find_all('section')[0])

    dict = {'2024' : [row[1] for row in info],
            '2023' : [row[2] for row in info],
            '2022' : [row[3] for row in info]}

    return pd.DataFrame(dict, index=[row[0] for row in info])

In [108]:
def get_skill_stats_df(soup):
    info = get_info_from_section(soup.find_all('section')[5])
    
    dict = {'Percentage' : [row[1] for row in info],
            'Skill' : [row[2] for row in info]}

    return pd.DataFrame(dict, index=[row[0] for row in info])

In [115]:
def get_category_stats_df(soup, i):
    info = get_info_from_section(soup.find_all('table')[6].find_all('table')[i])
    
    dict = {'Percentage' : [row[1] for row in info],
            'Skill' : [row[2] for row in info]}

    arrays = []
    return pd.DataFrame(dict, index=[row[0] for row in info])

In [125]:
df_genr = get_category_stats_df(soup, 7)
df_jobs = get_category_stats_df(soup, 8)
df_libs = get_category_stats_df(soup, 9)
df_lang = get_category_stats_df(soup, 13)

        Percentage         Skill
1   2,563 (33.54%)           SQL
2   2,122 (27.77%)          Java
3   1,454 (19.03%)    JavaScript
4   1,132 (14.81%)          Bash
5   1,017 (13.31%)            C#
6     795 (10.40%)    PowerShell
7      636 (8.32%)           C++
8      501 (6.56%)            Go
9      481 (6.29%)             C
10     477 (6.24%)             R
11     471 (6.16%)    TypeScript
12     414 (5.42%)          Ruby
13     369 (4.83%)          Perl
14     210 (2.75%)         Scala
15     184 (2.41%)           PHP
16     165 (2.16%)        MATLAB
17     138 (1.81%)            VB
18     137 (1.79%)          Rust
19     136 (1.78%)  Shell Script
20     110 (1.44%)         T-SQL
